# QA using wiki dataset

Thanks to Itsada (Por) <br>
tinyurl.com/ue3w86w3

In [ ]:
#load AIFORTHAI-ThaiQACorpus.zip
!gdown --id 1suaumoRn0inQORyYH-f2dRr0tbg2AMCX
!unzip -qq 'AIFORTHAI-ThaiQACorpus.zip'

#load wiki-documents.zip
!gdown --id 1SnD7WFwkI_cObHRySPZ593X_53J9T5Pv
!unzip -qq 'wiki-documents.zip'

Downloading...
From: https://drive.google.com/uc?id=1suaumoRn0inQORyYH-f2dRr0tbg2AMCX
To: /content/AIFORTHAI-ThaiQACorpus.zip
100% 267k/267k [00:00<00:00, 35.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1SnD7WFwkI_cObHRySPZ593X_53J9T5Pv
To: /content/wiki-documents.zip
178MB [00:03, 58.8MB/s]


In [ ]:
import os
import pandas as pd

In [ ]:
#create a dataframe named wiki_documents
wiki_documents = pd.DataFrame(sorted(os.listdir('wiki-documents')), columns=['file_name'])
wiki_documents['article_id'] = wiki_documents['file_name'].map(lambda x: x.replace('.txt', '')).astype(int)
wiki_documents['path'] = wiki_documents['file_name'].map(lambda x: os.path.join('wiki-documents', x))


In [ ]:
print(f'total rows: {wiki_documents.shape[0]}')
wiki_documents.tail()

total rows: 53654


,file_name,article_id,path
53649,430653.txt,430653,wiki-documents/430653.txt
53650,430667.txt,430667,wiki-documents/430667.txt
53651,430676.txt,430676,wiki-documents/430676.txt
53652,430687.txt,430687,wiki-documents/430687.txt
53653,43069.txt,43069,wiki-documents/43069.txt


In [ ]:
!pip install tqdm==4.60.0

In [ ]:
from tqdm.notebook import tqdm 

In [ ]:
#create a context column by reading from documents
for doc in tqdm(wiki_documents.itertuples(index=True), total=wiki_documents.shape[0]):
    f = open(doc.path, mode='r', encoding='utf-8')
    try:
        wiki_documents.loc[doc.Index, 'context'] = f.read()
    
    #some articles are unable to read due to decoding error
    except UnicodeDecodeError as e:
        print(str(e), doc.Index)
        wiki_documents.loc[doc.Index, 'context'] = None
    f.close()

wiki_documents = wiki_documents.loc[~wiki_documents['context'].isna()]
wiki_documents.reset_index(inplace=True, drop=True)
print(f'total rows: {wiki_documents.shape[0]}')
wiki_documents.tail()

  0%|          | 0/53654 [00:00<?, ?it/s]

In [ ]:
#since the document content contain HTML syntax, we have to remove it
def remove_HTML_syntax(input_text):
    text = ''
    accumulate = False
    output_text = input_text
    for char in input_text:
        if char == '<':
            accumulate = True
        elif char == '>':
            output_text = output_text.replace(text + char, '')
            text = ''
            accumulate = False
        
        if accumulate:
            text += char

    return output_text

#example function usage
text = wiki_documents.loc[0, 'context']
print(text)
remove_HTML_syntax(text)

In [ ]:
wiki_documents['context'] = wiki_documents['context'].map(lambda x: remove_HTML_syntax(x))
wiki_documents.head()

In [ ]:
import json

In [ ]:
with open('ThaiQACorpus-DevelopmentDataset.json') as file:
    qa_dataset = json.load(file)
qa_dataset.keys()

In [ ]:
print(len(qa_dataset['data']))
qa_dataset['data'][:5]

In [ ]:
qas = pd.DataFrame(qa_dataset['data'])
qas.head()

In [ ]:
import numpy as np
np.random.seed(0)

In [ ]:
#test train eval split
r = np.random.RandomState(2021)
split = r.rand(qas.shape[0])
qas.loc[split < 0.7, 'set'] = 'train'
qas.loc[(0.7 < split) & (split < 0.8), 'set'] = 'eval'
qas.loc[0.8 < split, 'set'] = 'test'

In [ ]:
qas['set'].value_counts()

In [ ]:
needed_article_ids = list(set([row.article_id for row in qas.itertuples()])) #make a unique list
needed_article_ids[:10]

In [ ]:
#verify which document is used
for doc in tqdm(wiki_documents.itertuples(index=True), total=wiki_documents.shape[0]):
    if doc.article_id in needed_article_ids:
        wiki_documents.loc[doc.Index, 'use'] = True
    else:
        wiki_documents.loc[doc.Index, 'use'] = False
wiki_documents.head()

In [ ]:
wiki_documents['use'].value_counts()

In [ ]:
needed_wiki_documents = wiki_documents.loc[wiki_documents['use'] == True]
needed_wiki_documents.shape[0]

In [ ]:
needed_wiki_documents.head()

In [ ]:
#make a SQUAD2.0 format dataset (visit: https://simpletransformers.ai/docs/qa-data-formats/)
question_id = 1
unknow_where_to_start = 0
dataset, train, eval, test = [], [], [], []
#assume one doc has just one article
for article in tqdm(needed_wiki_documents.itertuples(), total=needed_wiki_documents.shape[0]):
    document = {}
    document['context'] = article.context
    document['qas'] = [] 
    for qa in tqdm(qas.itertuples(), total=qas.shape[0], leave=False):
        if article.article_id == qa.article_id:
            question = {}
            question['id'] = question_id
            question_id += 1
            question['question'] = qa.question
            answer = {} #one question has just one answer
            answer['text'] = qa.answer 
            
            try:
                answer['answer_start'] = document['context'].index(answer['text'])
                question['is_impossible'] = False
            
            except:
                answer['answer_start'] = -1
                question['is_impossible'] = True
                unknow_where_to_start += 1
            
            question['answers'] = []
            question['answers'].append(answer)
            document['qas'].append(question)
            
            #may cause overhead while training, means same article, different question -> read twice
            if qa.set == 'train':
                train.append(document)
            elif qa.set == 'eval':
                eval.append(document)
            else:
                test.append(document)
    dataset.append(document)

In [ ]:
def reset_ids(dataset):
    id = 1
    for doc in dataset:
        for qa in doc['qas']:
            qa['id'] = id
            id += 1
    return dataset

train, eval, test = reset_ids(train), reset_ids(eval), reset_ids(test)

In [ ]:
eval[:2]

In [ ]:
with open('train.json', 'w', encoding='utf-8') as file:
  json.dump(train, file, ensure_ascii=False)
with open('eval.json', 'w', encoding='utf-8') as file:
  json.dump(eval, file, ensure_ascii=False)
with open('test.json', 'w', encoding='utf-8') as file:
  json.dump(test, file, ensure_ascii=False)

---

# Fine-tuning model

In [ ]:
import json

In [ ]:
with open('train.json', 'r') as file:
  train = json.load(file, encoding='utf-8')
with open('eval.json', 'r') as file:
  eval = json.load(file, encoding='utf-8')
with open('test.json', 'r') as file:
  test = json.load(file, encoding='utf-8')

In [ ]:
!pip install -qq transformers
!pip install -qq simpletransformers

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger('transformers')
transformers_logger.setLevel(logging.WARNING)

In [ ]:
import torch
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

torch.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [ ]:
# visit https://simpletransformers.ai/docs/usage/

model_args = {
    'manual_seed' : 0,
    'reprocess_input_data': False,
    
    'train_batch_size': 32,
    'eval_batch_size' : 8,
    
    'use_early_stopping': True,
    'early_stopping_delta': 0.01,
    'early_stopping_metric': 'eval_loss',
    'early_stopping_metric_minimize': True,
    'early_stopping_patience': 5,
    
    'evaluate_during_training' : True,
    'evaluate_during_training_verbose': True,
    
    'fp16': True,
    
    'num_train_epochs': 5,
    
    'overwrite_output_dir': True,

    'save_model_every_epoch': False,
    'save_steps':-1,
    
    'use_cached_eval_features' : True, 
    
    'max_seq_length': 256,
    'no_cache': False,

    'custom_parameter_groups':[{
        'params': ['classifier.weight', 'bert.encoder.layer.10.output.dense.weight'],
        'lr': 5e-5,
    }]
}

In [ ]:
#we can select a pretrained --> https://huggingface.co/models
#examples:
# 'monsoon-nlp/bert-base-thai'
# 'Geotrend/bert-base-th-cased'

In [ ]:
bert = QuestionAnsweringModel(
    'bert', 
    'monsoon-nlp/bert-base-thai',
    use_cuda=torch.cuda.is_available(), 
    args=model_args
)

In [ ]:
step, train_result = bert.train_model(train, eval_data=eval, show_running_loss=True)

In [ ]:
eval_result, eval_texts = bert.eval_model(eval)

In [ ]:
eval_result

In [ ]:
eval_texts.keys()

In [ ]:
eval_texts['correct_text']

In [ ]:
with open('test.json', 'r') as file:
  test = json.load(file, encoding='utf-8')

In [ ]:
test[0]

In [ ]:
#predict model (input should be: https://simpletransformers.ai/docs/qa-data-formats/#prediction-data-format) 
answers, probabilities = bert.predict(test)

In [ ]:
answers[0]

In [ ]:
probabilities[0]

In [ ]:
!zip -r 'best_model.zip' 'outputs/best_model' 

In [ ]:
from google.colab import files
files.download('best_model.zip') 

In [ ]:
# !cp '/content/drive/MyDrive/SuperAI/wiki-qa-dataset/best_model.zip' '.'

In [ ]:
# !mkdir saved_model

In [ ]:
# !unzip '/content/best_model.zip' -d 'saved_model'

In [ ]:
#load model
model = QuestionAnsweringModel(
    'bert', 
    # 'saved_model/outputs/best_model',
    use_cuda=torch.cuda.is_available()
    )

In [ ]:
arg1, arg2 = model.eval_model(test)

In [ ]:
arg1

In [ ]:
# arg2

#pretrained deepset/xlm-roberta-large-squad2
Thanks to P' Chomtana (Pin) to provide thorough guideline <br>
[link here](https://neoshare.net/machine-learning/solution-%E0%B8%97%E0%B8%B5%E0%B9%88%E0%B8%8A%E0%B8%99%E0%B8%B0%E0%B8%81%E0%B8%B2%E0%B8%A3%E0%B9%81%E0%B8%82%E0%B9%88%E0%B8%87%E0%B8%82%E0%B8%B1%E0%B8%99-super-ai-scg-thai-qa/)

In [ ]:
!pip install transformers
!pip -q install sentencepiece
!pip install tqdm==4.60.0
!pip install torch torchvision

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, AutoModelForSequenceClassification
import numpy as np
import re
import os
import pandas as pd
from tqdm.notebook import tqdm
import torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
torch.cuda.empty_cache()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

qa_name = "deepset/xlm-roberta-large-squad2"
yn_name = "/content/drive/MyDrive/qahack2021/model"
# qa_model = "deepset/roberta-base-squad2" no multilingual
model = AutoModelForQuestionAnswering.from_pretrained(qa_name)
yn_model = AutoModelForSequenceClassification.from_pretrained(yn_name)
model.to(device)
yn_model.to(device)
tokenizer = AutoTokenizer.from_pretrained(qa_name)

Some weights of the model checkpoint at /content/drive/MyDrive/qahack2021/model were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


###functions

In [ ]:
def use_model_multiline(question, contexts):
  length = len(contexts)
  lines = contexts.split('\n')
  answers = []

  for context in tqdm(lines, desc="multiline({})".format(str(length)) ):
    answers.append(use_model(question, context.strip(), already_use_multiline=True))
  
  maxscore = 0
  maxi = -1

  for i in range(len(answers)):
    try:
      answer = answers[i]
      if answer[0] and answer[0] != '<UNK/>' and answer[0] != '<s>' and answer[1] > maxscore:
        maxscore = answer[1]
        maxi = i
    except Exception as e:
      # print(e)
      continue

  if maxi == -1: return "<UNK/>", -1

  return answers[maxi]

In [ ]:
def use_modelv2(question, context, already_use_multiline=False):
  encoding = tokenizer(question, context, add_special_tokens=True, return_tensors="pt", truncation=True, padding=True).to(device)
  input_ids = encoding["input_ids"].detach().cpu().numpy()[0]

  answer = '<s>'
  iteration = 0

  model_ans = model(**encoding)

  answer_start_scores = model_ans.start_logits.detach().cpu().numpy()[0]
  answer_end_scores = model_ans.end_logits.detach().cpu().numpy()[0]
  
  while answer == '<s>' and iteration < 5:
    #print(answer_start_scores)
    answer_start = np.argmax(answer_start_scores)  
    answer_score = np.max(answer_start_scores) + np.max(answer_end_scores)
    answer_end_scores_filtered = answer_end_scores[answer_start:]

    if len(answer_end_scores_filtered) == 0:
      if answer_score == -99999999: return "<UNK/>", -1

      #print(answer_start, len(answer_start_scores))

      #set to -inf and find another one
      answer = '<s>'
      answer_start_scores[answer_start] = -99999999
      iteration += 1
      
      continue

    answer_end = np.argmax(answer_end_scores_filtered) + 1 + answer_start

    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

    #check answer is blank space (no answer)
    if answer.strip() and answer.strip() != '<s>':

      if len(answer.strip()) > 50 and not already_use_multiline:
        # try multiline context ensemble
        return use_model_multiline(question, context)
        
      else:
        return answer.strip(),  answer_score
    else:
      if answer_score == -99999999: return "<UNK/>", -1

      answer = '<s>'

      answer_start_scores[answer_start] = -99999999
      answer_end_scores[answer_end - 1] = -99999999

      iteration += 1
  
  return "<UNK/>", -1

In [ ]:
 def use_model(question, context, already_use_multiline=False):
  encoding = tokenizer(question, context, add_special_tokens=True, return_tensors="pt", truncation=True, padding=True).to(device)
  input_ids = encoding["input_ids"].detach().cpu().numpy()[0]
  try:
    answer = '<s>'
    iteration = 0

    model_ans = model(**encoding)

    answer_start_scores = model_ans.start_logits.detach().cpu().numpy()[0]
    answer_end_scores = model_ans.end_logits.detach().cpu().numpy()[0]
    
    while answer == '<s>' and iteration < 5:
      #print(answer_start_scores)
      answer_start = np.argmax(answer_start_scores)  
      answer_score = np.max(answer_start_scores) + np.max(answer_end_scores)
      answer_end_scores_filtered = answer_end_scores[answer_start:]

      if len(answer_end_scores_filtered) == 0:
        if answer_score == -99999999: return "<UNK/>", -1

        #print(answer_start, len(answer_start_scores))

        #set to -inf and find another one
        answer = '<s>'
        answer_start_scores[answer_start] = -99999999
        iteration += 1
        
        continue

      answer_end = np.argmax(answer_end_scores_filtered) + 1 + answer_start

      answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

      #check answer is blank space (no answer)
      if answer.strip() and answer.strip() != '<s>':
        return answer.strip(),  answer_score
      else:
        if answer_score == -99999999: return "<UNK/>", -1

        answer = '<s>'

        answer_start_scores[answer_start] = -99999999
        answer_end_scores[answer_end - 1] = -99999999

        iteration += 1
    
    return "<UNK/>", -1
  except Exception as e:
    # print(e)

    # try multiline context ensemble
    if already_use_multiline:
      return "<UNK/>", -1
    else:
      return use_model_multiline(question, context)
    

In [ ]:
def use_yn_model(question, context, max_length=416):
    # print(question)
    # print(context)
    sequence = tokenizer(question, context, return_tensors="pt", max_length=max_length, truncation_strategy="longest_first")['input_ids'].to(device)
    logits = yn_model(sequence)[0]
    probabilities = torch.softmax(logits, dim=1).detach().cpu().tolist()[0]
    prediction = np.array(probabilities).argmax()
    return "ใช่" if int(prediction) == 1 else "ไม่ใช่" # 1->Yes, 0-> NO

In [ ]:
#since the document content contain HTML syntax, we have to remove it
def remove_HTML_syntax(input_text):
    text = ''
    accumulate = False
    output_text = input_text
    for char in input_text:
        if char == '<':
            accumulate = True
        elif char == '>':
            output_text = output_text.replace(text + char, '')
            text = ''
            accumulate = False
        
        if accumulate:
            text += char

    return output_text

CUTTER = 400
def cut_to_lines(text):
  length = len(text)
  if length > CUTTER:
    marker = 0
    for i in range(1,length//CUTTER+1):
      
      marker = i*CUTTER
      while marker < (i+1)*CUTTER and marker < length - 1:
        if text[marker] == " ":
          break
        # print(marker, text[marker])
        marker += 1
      text = text[:marker] + "\n" + text[marker + 1:]
  return text

# def clean_blank_branket(text):
#   return re.sub(r'\(;* *\)',"", text)

def clean_text(text):
  text = remove_HTML_syntax(text)
  text = cut_to_lines(text)
  text = re.sub(r'\(;* *\)',"", text) #clear blank branket
  text = re.sub(r'[^A-Za-z0-9\u0E00-\u0E7F \n\u0021-\u005F\u0061-\u007E“”–]', '', text).strip()
  text = re.sub(' +', ' ', text) #clear multiple space
  return text

def clean_dup(arr):
  if len(arr) == 0: return []
  res = [arr[0]]
  for i in range(1, len(arr)):
    if arr[i] not in arr[:i]:
      res.append(arr[i])
  return res

def get_hashtags(text):
  tokens = text.split()
  hashtags = []

  for token in tokens:
    if len(token) > 0 and token[0] == '#':
      hashtags.append(token)

  return hashtags

def get_parts(text):
  tokens = text.split()

  hashtags = []
  years = []
  urls = []

  for i in range(len(tokens)):
    token = tokens[i]

    if len(token) > 0 and token[0] == '#':
      hashtags.append(token)

    if token.startswith('http'):
      urls.append(token)

    try:
      if abs(int(token) - 2020) <= 3 or abs(int(token) - 2563) <= 3:
        years.append(token)
    except:
      pass

    if 'ปีนี้' in token:
      years.append('ปีนี้')
  
  years = clean_dup(years)

  return hashtags, years, urls

###inputs

In [ ]:
if not "wiki-documents-nsc" in os.listdir("/content"):
  !unzip "/content/drive/MyDrive/qahack2021/wiki-documents-nsc.zip"

In [ ]:
#Small Test
# df = pd.read_csv("/content/drive/MyDrive/qahack2021/SmallTest.csv")
# df = df[["id", "article_id", "question"]]

#SCG-Train.csv
# df = pd.read_csv("/content/drive/MyDrive/qahack2021/SCG-Train.csv")
# df = df[["article_path", "question"]]
# df.reset_index(inplace=True)
# df.rename(columns={"article_path":"article_id", "index":"id"},inplace=True)

#Test
df = pd.read_csv("/content/drive/MyDrive/qahack2021/Test.csv")
df = df[["id", "article_id", "question"]]

df


,id,article_id,question
0,1,220726,ใครเป็นผู้เรียบเรียงเนื้อร้องเพลงคืนความสุขให้...
1,2,8217,การจัดนิทรรศการพันธุ์แมวในประเทศอังกฤษเมื่อปี ...
2,3,35303,ชาวอินเดียเรียกหมากรุกว่าอะไร
3,4,12493,จิ้งจกเป็นสัตว์เลื้อยคลานที่อยู่ในสกุลใด
4,5,11209,อะไรคือจุดเด่นเฉพาะตัวของเสือที่สัตว์อื่นไม่สา...
...,...,...,...
5603,5604,SCG217,จากบทความ ผู้เขียนร่วมงานกับ DO เป็นระยะเวลาเท...
5604,5605,SCG217,เรื่องอะไรที่ผู้เขียนได้พบว่าเป็นความสนใจใหม่?
5605,5606,SCG217,มี Project อะไรบ้างที่ผู้เขียนต้องทำงานพร้อมกัน?
5606,5607,SCG217,ผู้เขียนมองว่าเทคนิคที่สำคัญในการร่วมงานกับ Pa...


###preprocess

In [ ]:
def id_to_path(id):
  if "SCG" in id:
    dir = "/content/drive/MyDrive/qahack2021/test"
  elif "train" in id:
    dir = "/content/drive/MyDrive/qahack2021/train"
    id = "SCG" + id[6:9]
  else:
    dir = "/content/wiki-documents-nsc"
  return os.path.join(dir, str(id)+".txt")

In [ ]:
context = []
for row in df.itertuples():
  context.append(clean_text(open(id_to_path(row.article_id)).read()))
df["context"] = context
df

,id,article_id,question,context
0,1,220726,ใครเป็นผู้เรียบเรียงเนื้อร้องเพลงคืนความสุขให้...,ประยุทธ์ จันทร์โอชา พลเอก ประยุทธ์ จันทร์โอชา ...
1,2,8217,การจัดนิทรรศการพันธุ์แมวในประเทศอังกฤษเมื่อปี ...,แมว แมว เป็นสัตว์เลี้ยงลูกด้วยนม อยู่ในตระกูล ...
2,3,35303,ชาวอินเดียเรียกหมากรุกว่าอะไร,หมากรุก หมากรุก เป็นเกมกระดานชนิดหนึ่ง มีลักษณ...
3,4,12493,จิ้งจกเป็นสัตว์เลื้อยคลานที่อยู่ในสกุลใด,จิ้งจกบ้าน จิ้งจกบ้าน หรือที่นิยมเรียกสั้น ๆ ว...
4,5,11209,อะไรคือจุดเด่นเฉพาะตัวของเสือที่สัตว์อื่นไม่สา...,เสือ เสือ เป็นสัตว์เลี้ยงลูกด้วยนมในวงศ์ฟิลิดี...
...,...,...,...,...
5603,5604,SCG217,จากบทความ ผู้เขียนร่วมงานกับ DO เป็นระยะเวลาเท...,เมื่อมาทำงานจริงกับ Digitally Transforming Org...
5604,5605,SCG217,เรื่องอะไรที่ผู้เขียนได้พบว่าเป็นความสนใจใหม่?,เมื่อมาทำงานจริงกับ Digitally Transforming Org...
5605,5606,SCG217,มี Project อะไรบ้างที่ผู้เขียนต้องทำงานพร้อมกัน?,เมื่อมาทำงานจริงกับ Digitally Transforming Org...
5606,5607,SCG217,ผู้เขียนมองว่าเทคนิคที่สำคัญในการร่วมงานกับ Pa...,เมื่อมาทำงานจริงกับ Digitally Transforming Org...


###predict

In [ ]:
def save_data(name):
  sub_df = pd.DataFrame.from_dict(answers,orient='index',columns=["answer"])
  sub_df = sub_df.reset_index()
  sub_df.rename(columns={"index":"id"}, inplace=True)
  sub_df.sort_values(by=['id'], inplace=True)
  sub_df.to_csv('realsubmission{}.csv'.format(name), index=False)
  return sub_df

In [ ]:
answers = {
    1:"วิเชียร ตันติพิมลพันธุ์",
    2:"แมวเปอร์เซียและแมวขนสั้น",
    3:"จตุรังกา"
}
def assign_answer(id, ans):
  print("{}. {}".format(id, ans))
  if id not in answers.keys():
    answers[id] = ans

for row in df.itertuples():
  context = row.context
  hashtags, years, urls = get_parts(context)

  question = row.question
  id = row.id
  lowerquestion = question.lower()

  #yes/no
  if "ใช่หรือไม่" in question or "หรือไม่" in question:
      model_answer = use_yn_model(question, context)
      assign_answer(id, model_answer)
      
  # Hashtags
  elif ('hashtag' in question.lower() or 'แฮชแท็ก' in question) and ('ส่วนขยาย' not in question):
    question_hashtags = get_hashtags(question)
    answer_hashtags = []
    for hashtag in hashtags:
      if hashtag not in question_hashtags:
        answer_hashtags.append(hashtag)
    assign_answer(id, ' '.join(answer_hashtags))

  # ทางอีเมล
  elif 'ทางอีเมล' in context and 'ตรวจสอบ' in question and 'ที่ไหน' in question:
    assign_answer(id, 'ทางอีเมล')

  # Year?
  elif 'ปี' in question and len(years) > 0:
    assign_answer(id, years[0])
    years = years[1:]

  # URL
  elif len(urls) > 0 and (
      'url' in lowerquestion or 
      'website' in lowerquestion or 
      'download' in lowerquestion or
      'ดาวน์โหลด' in lowerquestion
  ):
    assign_answer(id, urls[0])
    urls = urls[1:]

  # QA model
  elif id not in answers.keys():
    try:
      model_answer, model_score = use_modelv2(question, context)
    except Exception:
      print("cublas error!")
      model_answer = "error"  


    # Fix quote and ()
    if '“' in model_answer and '”' not in model_answer:
      model_answer += '”'
    elif '”' in model_answer and '“' not in model_answer:
      model_answer = '“' + model_answer
    elif '(' in model_answer and ')' not in model_answer:
      model_answer += ')'
    elif ')' in model_answer and '(' not in model_answer:
      model_answer = '(' + model_answer

    index_in_context = context.find(model_answer)
    if index_in_context != -1 and index_in_context > 0 and index_in_context < len(context) - 1:
      if context[index_in_context-1] == '“' and context[index_in_context+len(model_answer)] == '”':
        model_answer = '“' + model_answer + '”'

    # Clean answer
    model_answer = model_answer.replace("<s>", "").replace("</s>", "")

    assign_answer(id, model_answer)
  
  #save csv every 100 rows
  if (id+1)%1000==0:
    save_data(id)
    torch.cuda.empty_cache()
    # answers = {}


4. Hemidactylus


multiline(38209):   0%|          | 0/96 [00:00<?, ?it/s]

5. สามารถซ่อนเล็บไว้ในปลายนิ้วเท้าได้
6. ประธานาธิบดีแห่งสาธารณรัฐฝรั่งเศส


multiline(2844):   0%|          | 0/8 [00:00<?, ?it/s]

7. เอดัวร์ ฟีลิป


...

5596. เป้าหมายที่ถูกต้อง ตัววัดผลที่ถูกต้อง ในช่วงเวลาที่ถูกต้อง
5597. monitor และวัดผล
5598. นวัตกรรมต่อยอดส่วนใหญ่
5599. หาไอเดียทางธุรกิจใหม่ให้เจอ
5600. Problem Solution Fit
5601. 7
5602. Digitally Transforming Organization


multiline(3480):   0%|          | 0/50 [00:00<?, ?it/s]

5603.  หน่วยงานเป็นอย่างไร Perspective :ในหน่วยงานมีคนเยอะมาก
5604. 3 เดือน 20 วัน
5605. User Experience design (UX)


multiline(3480):   0%|          | 0/50 [00:00<?, ?it/s]

5606. 1. consulting firms เจ้าใหญ่และบริษัท technology top 5 ของโลก
5607. keep update อยู่เสมอ
5608. ทุกคนมีเป้าหมายเดียวกันที่อยากเห็นความสําเร็จ


In [ ]:
df[200:]

In [ ]:
# answers

In [ ]:
sub_df = save_data("last2")
sub_df

,id,answer
0,1,วิเชียร ตันติพิมลพันธุ์
1,2,แมวเปอร์เซียและแมวขนสั้น
2,3,จตุรังกา
3,4,Hemidactylus
4,5,สามารถซ่อนเล็บไว้ในปลายนิ้วเท้าได้
...,...,...
5603,5604,3 เดือน 20 วัน
5604,5605,User Experience design (UX)
5605,5606,1. consulting firms เจ้าใหญ่และบริษัท technolo...
5606,5607,keep update อยู่เสมอ
